In [1]:
import numpy as np

In [2]:
# Re-import randomly splitted data frames (equal sources for various tests)
import pandas as pd
df_part1 = pd.read_csv('df_part1.csv')
#df_part1.head()

In [3]:
# Drop unneccesary colums
df_part1 = df_part1.drop(['Failure Type', 'Target', 'Type', 'Product ID', 'UDI', 'Unnamed: 0'], axis=1)
#df_part1.head()

In [4]:
# Split target values in separate columns
df_part1['Heat Dissipation Failure'] = df_part1['Failure Type Cat'].apply(lambda x: 1 if x == 0 else 0)
df_part1['No Failure'] = df_part1['Failure Type Cat'].apply(lambda x: 1 if x == 1 else 0)
df_part1['Overstrain Failure'] = df_part1['Failure Type Cat'].apply(lambda x: 1 if x == 2 else 0)
df_part1['Power Failure'] = df_part1['Failure Type Cat'].apply(lambda x: 1 if x == 3 else 0)
df_part1['Random Failures'] = df_part1['Failure Type Cat'].apply(lambda x: 1 if x == 4 else 0)
df_part1['Tool Wear Failure'] = df_part1['Failure Type Cat'].apply(lambda x: 1 if x == 5 else 0)
#df_part1

In [5]:
# Split dataset into train and testdata
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df_part1, train_size=0.8)
#df_train.shape

In [6]:
# Define feature and target names
feature_names = ['Air temperature [K]','Process temperature [K]','Rotational speed [rpm]','Torque [Nm]','Tool wear [min]']
target_names = ['Heat Dissipation Failure', 'No Failure', 'Overstrain Failure', 'Power Failure', 'Random Failures', 'Tool Wear Failure']

In [7]:
import tensorflow as tf

2023-08-23 22:12:58.484846: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
# Convert features for training to tensor
features_train = df_train[feature_names]
tf.convert_to_tensor(features_train)

<tf.Tensor: shape=(2666, 5), dtype=float64, numpy=
array([[ 298.3,  309.9, 1507. ,   41.6,  112. ],
       [ 302.2,  311.4, 1987. ,   19.9,  200. ],
       [ 300.8,  312.5, 1423. ,   47.3,  123. ],
       ...,
       [ 302.5,  310.2, 1586. ,   32.2,   64. ],
       [ 298.9,  309.9, 1645. ,   31.4,   86. ],
       [ 302.2,  311.1, 1553. ,   34.1,   89. ]])>

In [9]:
# Convert features for testing to tensor
features_test = df_test[feature_names]
tf.convert_to_tensor(features_test)

<tf.Tensor: shape=(667, 5), dtype=float64, numpy=
array([[ 303.2,  311.4, 1349. ,   51.1,    9. ],
       [ 301.3,  310.6, 1527. ,   44. ,  114. ],
       [ 296.8,  307.9, 1489. ,   33.4,  123. ],
       ...,
       [ 299.8,  310.2, 1654. ,   32.7,  159. ],
       [ 300.4,  310. , 1581. ,   30.3,   42. ],
       [ 300.8,  312.3, 1435. ,   44.8,  134. ]])>

In [10]:
# Convert target for training to tensor
target_train = df_train[target_names]
#tf.convert_to_tensor(target_train)

In [11]:
# Convert target for testing to tensor
target_test = df_test[target_names]
#tf.convert_to_tensor(target_test)

In [12]:
# Create a model and compile it
from tensorflow.keras.layers import Dense,BatchNormalization,Dropout,Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.regularizers import l2

In [13]:
def create_model():
    # 5 Features
    inputs=Input(5)
    # Layer with 500 neurons, rectified linear unit is simple and effective dealing with non-linearities,
    # kernel regularizer to prevent overfitting (good on training data, problem with new data)
    x=Dense(5000,activation='relu',kernel_regularizer=l2(0.01))(inputs)
    x=Dense(2500,activation='relu')(x)
    # Batch normalization to stabilize and speed up training
    x=BatchNormalization()(x)
    x=Dense(1250,activation='relu')(x)
    # Sigmoid activation for output nodes with one value zero
    x=Dense(6,activation='sigmoid')(x)
    model=Model(inputs,x)
    # CategoricalCrossentropy for multiclass, metrics accuracy, Adam = optimization algorithm providing:
    # adaptive learning, use of momentum and bias correction
    model.compile(loss=CategoricalCrossentropy(),metrics=['accuracy'],optimizer=Adam(0.01))
    return model

model=create_model()

In [14]:
model.fit(features_train, target_train, epochs=20, batch_size=100)

Epoch 1/20
27/27 [==============================] - 5s 141ms/step - loss: 9.4459 - accuracy: 0.8713
Epoch 2/20
27/27 [==============================] - 4s 150ms/step - loss: 5.7248 - accuracy: 0.9287
Epoch 3/20
27/27 [==============================] - 4s 167ms/step - loss: 2.9834 - accuracy: 0.9295
Epoch 4/20
27/27 [==============================] - 4s 144ms/step - loss: 1.3867 - accuracy: 0.9565
Epoch 5/20
27/27 [==============================] - 4s 136ms/step - loss: 0.9430 - accuracy: 0.9557
Epoch 6/20
27/27 [==============================] - 4s 138ms/step - loss: 0.7991 - accuracy: 0.9587
Epoch 7/20
27/27 [==============================] - 4s 140ms/step - loss: 0.6683 - accuracy: 0.9625
Epoch 8/20
27/27 [==============================] - 4s 137ms/step - loss: 0.5996 - accuracy: 0.9625
Epoch 9/20
27/27 [==============================] - 4s 135ms/step - loss: 0.5465 - accuracy: 0.9632
Epoch 10/20
27/27 [==============================] - 4s 155ms/step - loss: 0.5141 - accuracy: 0.9629

In [15]:
# Predict behaviour based on trained model
import numpy as np
target_pred = model.predict(features_test)
target_pred#[:10]

21/21 [==============================] - 0s 14ms/step


array([[1.0000000e+00, 1.0000000e+00, 9.6414953e-01, 1.0000000e+00,
        4.4975779e-17, 2.2990913e-11],
       [1.0000000e+00, 1.0000000e+00, 9.9997032e-01, 9.9996138e-01,
        8.0436301e-08, 7.1408985e-09],
       [1.0000000e+00, 1.0000000e+00, 9.9997830e-01, 9.9997938e-01,
        6.8583161e-09, 3.6459273e-09],
       ...,
       [1.0000000e+00, 1.0000000e+00, 9.9999303e-01, 9.9765217e-01,
        4.0994465e-04, 7.1695069e-08],
       [1.0000000e+00, 1.0000000e+00, 9.9924189e-01, 9.9999028e-01,
        3.7884561e-08, 2.5366749e-09],
       [1.0000000e+00, 1.0000000e+00, 9.9999171e-01, 9.9998730e-01,
        2.8267347e-10, 2.1068596e-09]], dtype=float32)

In [16]:
target_pred_final = pd.DataFrame(target_pred)
target_pred_final

,0,1,2,3,4,5
0,1.0,1.0,0.964150,1.000000,4.497578e-17,2.299091e-11
1,1.0,1.0,0.999970,0.999961,8.043630e-08,7.140899e-09
2,1.0,1.0,0.999978,0.999979,6.858316e-09,3.645927e-09
3,1.0,1.0,0.991001,1.000000,1.818047e-11,2.033514e-10
4,1.0,1.0,0.999747,0.999998,1.130672e-11,4.483667e-10
...,...,...,...,...,...,...
662,1.0,1.0,0.999879,0.999989,8.768057e-09,2.845366e-09
663,1.0,1.0,0.995630,1.000000,1.223176e-14,6.883800e-11
664,1.0,1.0,0.999993,0.997652,4.099447e-04,7.169507e-08
665,1.0,1.0,0.999242,0.999990,3.788456e-08,2.536675e-09


In [17]:
print("min:")
i = 0
for i in target_pred_final:
    print(min(target_pred_final[i]))
    i+1
print("max:")
i = 0
for i in target_pred_final:
    print(max(target_pred_final[i]))
    i+1

min:
1.0
1.0
0.3100421726703644
0.8045642971992493
1.0427076354967398e-22
1.0947110405667981e-11
max:
1.0
1.0
0.9999998807907104
1.0
0.9999979734420776
1.407296622346621e-05


In [18]:
target_pred_final[0] = target_pred_final[0].apply(lambda x: 0 if x < 0.5 else 1)
target_pred_final[1] = target_pred_final[1].apply(lambda x: 0 if x < 0.5 else 1)
target_pred_final[2] = target_pred_final[2].apply(lambda x: 0 if x < 0.5 else 1)
target_pred_final[3] = target_pred_final[3].apply(lambda x: 0 if x < 0.5 else 1)
target_pred_final[4] = target_pred_final[4].apply(lambda x: 0 if x < 0.5 else 1)
target_pred_final[5] = target_pred_final[5].apply(lambda x: 0 if x < 0.5 else 1)
target_pred_final

,0,1,2,3,4,5
0,1,1,1,1,0,0
1,1,1,1,1,0,0
2,1,1,1,1,0,0
3,1,1,1,1,0,0
4,1,1,1,1,0,0
...,...,...,...,...,...,...
662,1,1,1,1,0,0
663,1,1,1,1,0,0
664,1,1,1,1,0,0
665,1,1,1,1,0,0


In [20]:
print("min:")
i = 0
for i in target_pred_final:
    print(min(target_pred_final[i]))
    i+1
print("max:")
i = 0
for i in target_pred_final:
    print(max(target_pred_final[i]))
    i+1

min:
1
1
0
1
0
0
max:
1
1
1
1
1
0
